# 快速搭建 RAG Demo

1. 调用 DeepSeek R1

在该部分，你需要从任一 LLM 平台获取一个 DeepSeek R1 调用的接口，并参考平台的代码示例完成调用代码，此处仅给出几个推荐平台链接：
- DeepSeek：https://platform.deepseek.com/usage
- 火山：https://www.volcengine.com/docs/82379/
- 文心千帆：https://cloud.baidu.com/product-s/qianfan_home

In [1]:
# 安装 openai
!pip install openai

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:

'''此处可以测试调用 LLM API 的代码'''


'此处可以测试调用 LLM API 的代码'

In [4]:
# 安装 langchain
! pip install langchain==0.3.0
! pip install langchain-community==0.3.0
! pip install langchain-text-splitters==0.3.0
! pip install langchain-core==0.3.0
! pip install langchain-openai==0.2.0
! pip install langchain-chroma==0.1.4

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.4/602.4 kB 60.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.0 MB/s eta 0:00:00a 0:00:01
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.48
    Uninstalling langchain-core-0.3.48:
      Successfully uninstalled langchain-core-0.3.48
Looking in indexes:

In [ ]:
# 封装成 Langchain 可调用的类
'''需要根据使用的 LLM API 进行修改'''

from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun

class DeepSeekLLM(LLM):
    # 基于DeepSeek 接口自定义 LLM 类
    api_key : str = None
    model: str = None

    def __init__(self, api_key=None, model=None):
        # model：选择使用的模型
        # 从本地初始化模型
        super().__init__()
        self.api_key = api_key
        self.model = model


    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):
        # 重写调用函数
        '''在该方法中实现调用 LLM API 的逻辑'''
        response = get_response(prompt)    
        return response
        
    @property
    def _llm_type(self) -> str:
        return "DeepSeek R1"

In [ ]:
'''DEEPSEEK_API_KEY 是所调用 API 的 API Key'''
llm = DeepSeekLLM(api_key = DEEPSEEK_API_KEY)
llm.invoke("你是谁")

'<think>\n好的，用户问我“你是谁”，我需要详细思考一下如何回答。首先，我应该明确自己的身份，用户提到我是一个有用的助手，所以我的定义应该基于这个。我需要解释自己是一个AI助手，由深度求索公司开发，而不是真实存在的人。\n\n接下来，我应该说明我的主要功能，比如提供信息帮助、解答问题、提供建议等。同时，要强调我擅长的领域，如自然语言处理，能够处理多种任务，比如翻译、写作、编程等，这样用户能更好地了解我的能力范围。\n\n然后，我需要提到我的局限性，比如无法实时更新知识库，无法执行真实世界的操作，或者理解某些复杂的情感和上下文。这有助于用户明确我的能力边界，避免产生误解。\n\n此外，我应该鼓励用户随时提问，并简要说明如何更好地帮助他们，比如提供具体的问题或背景信息。这样可以引导用户更有效地利用我的服务。\n\n最后，保持回答的正式但又不失亲切，确保信息清晰易懂，同时避免使用过于技术化的术语，让用户容易理解。整个思考过程需要逻辑清晰，结构合理，确保回答全面而准确。\n</think>\n\n您好！我是由深度求索（DeepSeek）公司开发的智能助手DeepSeek-R1。如您有任何任何问题，我会尽我所能为您提供帮助。'

2. 调用 Embedding 模型

Embedding 模型可以通过以下两种方式：
- 使用 LLM 平台提供的 API，同上，可以自行在平台查找示例代码；
- 从 Huggingface 下载开源 Embedding 模型，然后进行本地部署，此处用本地部署开源 Embedding 模型作为示例

下载 embedding 模型：在终端 python download_model.py

In [11]:
!pip install accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [ ]:
# Requires transformers>=4.36.0

import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
import torch

input_texts = [
    "what is the capital of China?",
    "how to implement quick sort in python?",
    "北京",
    "快排算法介绍"
]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name_or_path = 'embedding_model_small'
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True, device_map="auto")

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt').to("cuda")
outputs = model(**batch_dict)

dimension=768 # The output dimension of the output embedding, should be in [128, 768]
embeddings = outputs.last_hidden_state[:, 0][:dimension]
print(embeddings[0].shape)

# [[0.3016996383666992, 0.7503870129585266, 0.3203084468841553]]


Some weights of the model checkpoint at embedding_model_small were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([768])


/root/.cache/huggingface/modules/transformers_modules/Alibaba-NLP/new-impl/40ced75c3017eb27626c9d4ea981bde21a2662f4/modeling.py:579: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at /data/jenkins_workspace/workspace/pytorch@4/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:459.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
/root/.cache/huggingface/modules/transformers_modules/Alibaba-NLP/new-impl/40ced75c3017eb27626c9d4ea981bde21a2662f4/modeling.py:579: UserWarning: 1Torch was not compiled with memory efficient attention. (Triggered internally at /data/jenkins_workspace/workspace/pytorch@4/aten/src/ATen/native/transformers/hip/sdp_utils.cpp:508.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [ ]:
# 封装成 langchain 可调用的类

from typing import List
from langchain_core.embeddings import Embeddings
import torch
from transformers import AutoModel, AutoTokenizer

class Embeddings(Embeddings):
    def __init__(self):
        """
        先加载模型
        """          
        # 模型下载的本地路径
        model_name_or_path = 'embedding_model'
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
        self.model = AutoModel.from_pretrained(model_name_or_path, trust_remote_code=True, device_map="auto")

    
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """
        生成输入文本列表的 embedding.
        Args:
            texts (List[str]): 要生成 embedding 的文本列表.

        Returns:
            List[List[float]]: 输入列表中每个文档的 embedding 列表。每个 embedding 都表示为一个浮点值列表。
        """

        result = []
        for i in range(0, len(texts), 16):
            input_texts = texts[i:i+16]
            batch_dict = self.tokenizer(input_texts, max_length=8192, padding=True, truncation=True, return_tensors='pt').to("cuda")
            outputs = self.model(**batch_dict)
            dimension=768 # The output dimension of the output embedding, should be in [128, 768]
            embeddings = outputs.last_hidden_state[:, 0][:dimension]
            result.extend([item.cpu().detach().numpy() for item in embeddings])
        return result
    
    def embed_query(self, text: str) -> List[float]:
        """
        生成输入文本的 embedding.

        Args:
            texts (str): 要生成 embedding 的文本.

        Return:
            embeddings (List[float]): 输入文本的 embedding，一个浮点数值列表.
        """
        batch_dict = self.tokenizer([text], max_length=8192, padding=True, truncation=True, return_tensors='pt').to("cuda")
        outputs = self.model(**batch_dict)
        dimension=768 # The output dimension of the output embedding, should be in [128, 768]
        embeddings = outputs.last_hidden_state[:, 0][:dimension]
        return embeddings[0].cpu().detach().numpy()

3. 知识文档处理


分别下载两个 pdf 文件：

- wget https://github.com/datawhalechina/llm-cookbook/releases/download/v1%2C0%2C0/LLM-v1.0.0.pdf
- wget https://github.com/datawhalechina/pumpkin-book/releases/download/v2.0.0/pumpkin_book.pdf

我们可以使用 LangChain 的 PyMuPDFLoader 来读取知识库的 PDF 文件。PyMuPDFLoader 是 PDF 解析器中速度最快的一种，结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。

In [11]:
!pip3 install pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 29.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
# 使用 langchain 自带的 loader 读取
from langchain_community.document_loaders import PyMuPDFLoader
import os

# 存储路径
pdf_file = "/pdf/LLM-v1.0.0.pdf"

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader(pdf_file)

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pdf_pages = loader.load()

文档加载后储存在 pages 变量中:

- page 的变量类型为 List
- 打印 pages 的长度可以看到 pdf 一共包含多少页

In [14]:
print(f"载入后的变量类型为：{type(pdf_pages)}，",  f"该 PDF 一共包含 {len(pdf_pages)} 页")

载入后的变量类型为：<class 'list'>， 该 PDF 一共包含 373 页


page 中的每一元素为一个文档，变量类型为 langchain_core.documents.base.Document, 文档变量类型包含两个属性

- page_content 包含该文档的内容。
- meta_data 为文档相关的描述性数据。

In [15]:
pdf_page = pdf_pages[10]
print(f"每一个元素的类型：{type(pdf_page)}.", 
    f"该文档的描述性数据：{pdf_page.metadata}", 
    f"查看该文档的内容:\n{pdf_page.page_content}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': '/root/private_data/zyh/data/base_data/pdf/LLM-v1.0.0.pdf', 'file_path': '/root/private_data/zyh/data/base_data/pdf/LLM-v1.0.0.pdf', 'page': 10, 'total_pages': 373, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20230817062022Z', 'trapped': ''}
------
查看该文档的内容:
第二章 提示原则
 
如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个
开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清
晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。
首先，Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型准确理解我们的意图，就像向一
个外星人详细解释人类世界一样。过于简略的 Prompt 往往使模型难以把握所要完成的具体任务。
其次，让语言模型有充足时间推理也极为关键。就像人类解题一样，匆忙得出的结论多有失误。因此 
Prompt 应加入逐步推理的要求，给模型留出充分思考时间，这样生成的结果才更准确可靠。
如果 Prompt 在这两点上都作了优化，语言模型就能够尽可能发挥潜力，完成复杂的推理和生成任务。
掌握这些 Prompt 设计原则，是开发者取得语言模型应用成功的重要一步。
一、原则一 编写清晰、具体的指令
 
亲爱的读者，在与语言模型交互时，您需要牢记一点:以清晰、具体的方式表达您的需求。假设您面前坐
着一位来自外星球的新朋友，其对人类语言和常识都一无所知。在这种情况下，您需要把想

由于单个文档的长度往往会超过模型支持的上下文，导致检索得到的知识太长超出模型的处理能力，因此，在构建向量知识库的过程中，我们往往需要对文档进行分割，将单个文档按长度或者按固定的规则分割成若干个 chunk，然后将每个 chunk 转化为词向量，存储到向量数据库中。

在检索时，我们会以 chunk 作为检索的元单位，也就是每一次检索到 k 个 chunk 作为模型可以参考来回答用户问题的知识，这个 k 是我们可以自由设定的。

Langchain 中文本分割器都根据 chunk_size (块大小)和 chunk_overlap (块与块之间的重叠大小)进行分割:

- chunk_size 指每个块包含的字符或 Token （如单词、句子等）的数量
- chunk_overlap 指两个块之间共享的字符数量，用于保持上下文的连贯性，避免分割丢失上下文信息

Langchain 提供多种文档分割方式，区别在怎么确定块与块之间的边界、块由哪些字符/token组成、以及如何测量块大小。这里我们使用最常用的分割器：

RecursiveCharacterTextSplitter(): 按字符串分割文本，递归地尝试按不同的分隔符进行分割文本。

In [16]:
''' 
* RecursiveCharacterTextSplitter 递归字符文本分割
RecursiveCharacterTextSplitter 将按不同的字符递归地分割(按照这个优先级["\n\n", "\n", " ", ""])，
    这样就能尽量把所有和语义相关的内容尽可能长时间地保留在同一位置
RecursiveCharacterTextSplitter需要关注的是4个参数：

* separators - 分隔符字符串数组
* chunk_size - 每个文档的字符数量限制
* chunk_overlap - 两份文档重叠区域的长度
* length_function - 长度计算函数
'''
#导入文本分割器
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 知识库中单段文本长度
CHUNK_SIZE = 500

# 知识库中相邻文本重合长度
OVERLAP_SIZE = 50

# 使用递归字符文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=OVERLAP_SIZE
)
text_splitter.split_text(pdf_page.page_content[0:1000])

['第二章 提示原则\n \n如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个\n开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清\n晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。\n首先，Prompt 需要清晰明确地表达需求，提供充足上下文，使语言模型准确理解我们的意图，就像向一\n个外星人详细解释人类世界一样。过于简略的 Prompt 往往使模型难以把握所要完成的具体任务。\n其次，让语言模型有充足时间推理也极为关键。就像人类解题一样，匆忙得出的结论多有失误。因此 \nPrompt 应加入逐步推理的要求，给模型留出充分思考时间，这样生成的结果才更准确可靠。\n如果 Prompt 在这两点上都作了优化，语言模型就能够尽可能发挥潜力，完成复杂的推理和生成任务。\n掌握这些 Prompt 设计原则，是开发者取得语言模型应用成功的重要一步。\n一、原则一 编写清晰、具体的指令',
 '一、原则一 编写清晰、具体的指令\n \n亲爱的读者，在与语言模型交互时，您需要牢记一点:以清晰、具体的方式表达您的需求。假设您面前坐\n着一位来自外星球的新朋友，其对人类语言和常识都一无所知。在这种情况下，您需要把想表达的意图\n讲得非常明确，不要有任何歧义。同样的，在提供 Prompt 的时候，也要以足够详细和容易理解的方\n式，把您的需求与上下文说清楚。 \n并不是说 Prompt 就必须非常短小简洁。事实上，在许多情况下，更长、更复杂的 Prompt 反而会让语\n言模型更容易抓住关键点，给出符合预期的回复。原因在于，复杂的 Prompt 提供了更丰富的上下文和\n细节，让模型可以更准确地把握所需的操作和响应方式。\n所以，记住用清晰、详尽的语言表达 Prompt，就像在给外星人讲解人类世界一样，“Adding more \ncontext helps the model understand you better.”。\n从该原则出发，我们提供几个设计 Prompt 的技巧。\n1.1 使用分隔符清晰地表示输入的不同部分',
 '1.1 使用分隔符清晰地表示输入的不同部分\n \n在编写 Prompt 时，我们可以使用各种标点

In [17]:
split_docs = text_splitter.split_documents(pdf_pages)
print(f"切分后的文件数量：{len(split_docs)}")

切分后的文件数量：1281


In [18]:
print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

切分后的字符数（可以用来大致评估 token 数）：523817


注：如何对文档进行分割，其实是数据处理中最核心的一步，其往往决定了检索系统的下限。但是，如何选择分割方式，往往具有很强的业务相关性——针对不同的业务、不同的源数据，往往需要设定个性化的文档分割方式。因此，在该部分，我们仅简单根据 chunk_size 对文档进行分割。后续在 RAG 应用的优化环节，我们会进一步探究如何优化文档的处理及分割从而提高 RAG 系统的检索精度和回答性能。

4. 构建向量知识库

In [5]:
# 获取 folder_path 下所有文件路径，储存在 file_paths 里
import os
file_paths = []
folder_path = 'data/base_data/pdf'
files = os.listdir(folder_path)
for file in files:
    # 忽略隐藏文件
    if file[0] == ".": continue
    file_path = os.path.join(folder_path, file)
    file_paths.append(file_path)
print(file_paths)

['data/base_data/pdf/LLM-v1.0.0.pdf', 'data/base_data/pdf/pumpkin_book.pdf']


In [6]:
from langchain_community.document_loaders import PyMuPDFLoader

# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))

len(loaders)

2

In [7]:
# 下载文件并存储到text
texts = []

for loader in loaders: 
    texts.extend(loader.load())

len(texts)

569

In [9]:
text = texts[1]
print(f"每一个元素的类型：{type(text)}.", 
    f"该文档的描述性数据：{text.metadata}", 
    f"查看该文档的内容:\n{text.page_content[0:]}", 
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'source': 'data/base_data/pdf/LLM-v1.0.0.pdf', 'file_path': 'data/base_data/pdf/LLM-v1.0.0.pdf', 'page': 1, 'total_pages': 373, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iLovePDF', 'creationDate': '', 'modDate': 'D:20230817062022Z', 'trapped': ''}
------
查看该文档的内容:
7. 文本扩展 Expanding @邹雨衡
8. 聊天机器人 Chatbot @长琴
9. 总结 @长琴
附1 使用 ChatGLM 进行学习 @宋志学
二、搭建基于 ChatGPT 的问答系统
 
 注：吴恩达《Building Systems with the ChatGPT API》课程中文版
 目录：
1. 简介 Introduction @Sarai
2. 模型，范式和 token Language Models, the Chat Format and Tokens @仲泰
3. 检查输入-分类 Classification @诸世纪
4. 检查输入-监督 Moderation @诸世纪
5. 思维链推理 Chain of Thought Reasoning @万礼行
6. 提示链 Chaining Prompts @万礼行
7. 检查输入 Check Outputs @仲泰
8. 评估（端到端系统）Evaluation @邹雨衡
9. 评估（简单问答）Evaluation-part1 @陈志宏、邹雨衡
10. 评估（复杂问答）Evaluation-part2 @邹雨衡
11. 总结 Conclusion @Sarai
三、使用 LangChain 开发应用程序
 
 注：吴恩达《LangChain for LLM Application Develop

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 切分文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(texts)
len(split_docs)

1995

In [9]:
# 使用 chroma 构建向量数据库
from langchain_community.vectorstores import Chroma

# 定义 Embeddings
embedding = Embeddings()

# 定义持久化路径
persist_directory = 'data/chroma'

vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

vectordb.persist()

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0 with CUDA None (you have 2.3.0)
    Python  3.11.9 (you have 3.11.9)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
Some weights of the model checkpoint at embedding_model_small were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/root/.cache/

In [10]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1995


In [22]:
# 持久化向量库
vectordb.persist()

/tmp/ipykernel_8369/2233374908.py:2: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [11]:
# 测试一下向量检索

question="什么是南瓜书"
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content}", end="\n--------------\n")

检索到的内容数：3
检索到的第0个内容: 
本:2.0.0
发布日期:2023.11
南  ⽠  书
PUMPKIN
B  O  O  K
谢	睿 
州 贾彬彬
--------------
检索到的第1个内容: 
最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases
编委会
主编：Sm1les、archwalker、jbb0523
编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226
封面设计：构思-Sm1les、创作-林王茂盛
致谢
特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、
spareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。
扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”
版权声明
本作品采用知识共享署名-非商业性使用-相同方式共享4.0 国际许可协议进行许可。
--------------
检索到的第2个内容: 
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我
等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二
下学生”。
使用说明
• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书
为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；
• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式

In [ ]:
# 封装成一个检索器
def get_retriever():
    # 定义 Embeddings
    embedding = Embeddings()
    # 向量数据库持久化路径
    persist_directory = 'data/chroma'
    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    return vectordb.as_retriever()

5. 构建 RAG 链路

使用 LangChain 构建 RAG 链路

In [12]:
# 先构建一个检索链
from langchain_core.runnables import RunnableLambda

def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever = get_retriever()
combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么？")

/tmp/ipykernel_2542/1517970404.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(


'前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n\n最新版PDF 获取地址：https://github.com/datawhalechina/pumpkin-book/releases\n编委会\n主编：Sm1les、archwalker、jbb0523\n编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226\n封面设计：构思-Sm1les、创作-林王茂盛\n致谢\n特别感谢awyd234、feijuan、Ggmatch、Heitao5200、huaqing89、LongJH、LilRachel、LeoLRH、Nono17、\nspareribs、sunchaothu、StevenLzq 在最早期的时候对南瓜书所做的贡献。\n扫描下方二维码，然后回复关键词“南瓜书”，即可加入“南瓜书读者交流群”\n版权声明\n本作品采用知识共享署名-非商业性使用-相同方式共享4.0 国际许可协议进行许可。\n\n\x01本\x03:2.0.0\n发布日期:2023.11\n南  ⽠  书\nPUMPKIN\nB  O  O  K\n谢\t睿 \x0b州 贾彬彬\n\n• 对于初学机器

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

template = """
参考给定的知识，回答用户的问题。
你可以结合给定知识进行适当推理，但务必准确。
回答用户问题时，你需要给出你的参考依据。
可参考的知识：
```
{context}
```
需要回答的用户问题: {input}
"""

# 将template通过 PromptTemplate 转为可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
# 测试效果

question_1 = "什么是南瓜书？"

result = qa_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)
print()

result = llm.invoke(question_1)
print("大模型直接回答 question_1 的结果：")
print(result)

大模型+知识库后回答 question_1 的结果：
<think>
好的，我需要回答用户的问题：“什么是南瓜书？”首先，我会查看提供的知识内容，寻找与南瓜书相关的信息。

在知识中，南瓜书被提到是周志华《机器学习》（西瓜书）的补充解析，专注于解析书中的公式和推导细节。编委会和编委的信息显示，南瓜书是由Sm1les、archwalker、jbb0523等主编，还有其他编委和贡献者。南瓜书的版权信息和获取方式也提到了GitHub地址和纸质版的购买渠道。

此外，南瓜书的使用说明建议用户以西瓜书为主线，遇到困难时查阅南瓜书。还提到了致谢部分，感谢早期贡献者，并说明南瓜书的内容是基于西瓜书的前置知识进行的。南瓜书还有配套的视频教程和在线阅读地址。

综合以上信息，南瓜书是一个开源项目，旨在帮助读者深入理解西瓜书中的公式和推导，适合数学基础较弱的自学读者。因此，回答应涵盖南瓜书的定义、内容、编委会、获取方式和使用建议等方面。
</think>

南瓜书是《机器学习公式详解》的别称，由国内社区DataWhale组织编写，旨在帮助读者深入理解周志华《机器学习》（西瓜书）中的公式和推导细节。以下是关于南瓜书的详细介绍：

1. **定义与目的**：
   - 南瓜书是对西瓜书中较难理解公式的解析和补充推导，特别适合数学基础较弱的读者。

2. **编委会与贡献者**：
   - 主编：Sm1les、archwalker、jbb0523。
   - 编委：juxiao、Majingmin、MrBigFan、shanry、Ye980226。
   - 其他贡献者包括awyd234、feijuan、Ggmatch等。

3. **获取方式**：
   - 最新版PDF可在GitHub获取：https://github.com/datawhalechina/pumpkin-book/releases。
   - 纸质版可在各大电商平台选购。

4. **使用说明**：
   - 以西瓜书为主线，遇到推导困难时查阅南瓜书。
   - 对于初学机器学习的小白，建议先简单过一遍西瓜书的前两章。

5. **版权与致谢**：
   - 使用知识共享署名-非商业性使用-相同方式共享4.0国际许可协议。
   - 感谢早期贡献者和社区支持。

6. **配套资源**：
   - 视频教程：B站视频B

6. 构建带历史记录的 RAG 链路

LangChain 中提供了带有历史记录的链路组件，可以通过 LLM 对历史记录进行压缩来支持历史记录

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# 问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。 "
    "请使用检索到的上下文片段回答这个问题。 "
    "如果你不知道答案就说不知道。 "
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)
# 制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

In [21]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )
# 构造压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ])
# 构造检索文档的链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

这里我们使用之前定义的问答模板即qa_prompt构造问答链，另外我们通过RunnablePassthrough.assign将中间的查询结果存为"context"，将最终结果存为"answer"。因为查询结果被存为"context"，所以我们整合查询结果的函数combine_docs也要做相应的改动。

In [22]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer

In [23]:
# 测试一下

# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20231117152045-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/base_data/pdf/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/base_data/pdf/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得'),
  Document(metadata={'author': '', 'creati

In [24]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})

{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20231117152045-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': 'data/base_data/pdf/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': 'data/base_data/pdf/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 

7. 部署前端界面

In [12]:
!pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 5.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 3.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 2.4 MB/s eta 0:00:0000:0100:01


In [31]:
import streamlit as st
from langchain_openai import ChatOpenAI
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableBranch, RunnablePassthrough
from langchain_community.vectorstores import Chroma

In [ ]:
# 1. 定义 get_retriever 函数，该函数返回一个检索器
def get_retriever():
    # 定义 Embeddings
    embedding = Embeddings()
    # 向量数据库持久化路径
    persist_directory = 'data/chroma'
    # 加载数据库
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    return vectordb.as_retriever()

In [26]:
# 2. 定义combine_docs函数， 该函数处理检索器返回的文本
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"])

In [27]:
# 3. 定义get_qa_history_chain函数，该函数可以返回一个检索问答链
def get_qa_history_chain():
    retriever = get_retriever()
    llm = llm = DeepSeekLLM(api_key = DEEPSEEK_API_KEY)
    condense_question_system_template = (
        "请根据聊天记录总结用户最近的问题，"
        "如果没有多余的聊天记录则返回用户的问题。"
    )
    condense_question_prompt = ChatPromptTemplate([
            ("system", condense_question_system_template),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ])

    retrieve_docs = RunnableBranch(
        (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
        condense_question_prompt | llm | StrOutputParser() | retriever,
    )

    system_prompt = (
        "你是一个问答任务的助手。 "
        "请使用检索到的上下文片段回答这个问题。 "
        "如果你不知道答案就说不知道。 "
        "请使用简洁的话语回答用户。"
        "\n\n"
        "{context}"
    )
    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("placeholder", "{chat_history}"),
            ("human", "{input}"),
        ]
    )
    qa_chain = (
        RunnablePassthrough().assign(context=combine_docs)
        | qa_prompt
        | llm
        | StrOutputParser()
    )

    qa_history_chain = RunnablePassthrough().assign(
        context = retrieve_docs, 
        ).assign(answer=qa_chain)
    return qa_history_chain

In [28]:
# 4. 定义gen_response函数，它接受检索问答链、用户输入及聊天历史，并以流式返回该链输出
def gen_response(chain, input, chat_history):
    response = chain.stream({
        "input": input,
        "chat_history": chat_history
    })
    for res in response:
        if "answer" in res.keys():
            yield res["answer"]

In [29]:
# 5. 定义main函数，该函数制定显示效果与逻辑
def main():
    st.markdown('### 🦜🔗 基于 DeepSeek R1 搭建 RAG 应用')
    # st.session_state可以存储用户与应用交互期间的状态与数据
    # 存储对话历史
    if "messages" not in st.session_state:
        st.session_state.messages = []
    # 存储检索问答链
    if "qa_history_chain" not in st.session_state:
        st.session_state.qa_history_chain = get_qa_history_chain()
    # 建立容器 高度为500 px
    messages = st.container(height=550)
    # 显示整个对话历史
    for message in st.session_state.messages: # 遍历对话历史
            with messages.chat_message(message[0]): # messages指在容器下显示，chat_message显示用户及ai头像
                st.write(message[1]) # 打印内容
    if prompt := st.chat_input("Say something"):
        # 将用户输入添加到对话历史中
        st.session_state.messages.append(("human", prompt))
        # 显示当前用户输入
        with messages.chat_message("human"):
            st.write(prompt)
        # 生成回复
        answer = gen_response(
            chain=st.session_state.qa_history_chain,
            input=prompt,
            chat_history=st.session_state.messages
        )
        # 流式输出
        with messages.chat_message("ai"):
            output = st.write_stream(answer)
        # 将输出存入st.session_state.messages
        st.session_state.messages.append(("ai", output))

将上述代码粘贴到一个 Python 文件，再通过 “streamlit run”运行即可